<a href="https://colab.research.google.com/github/sahilwakode/Tensorflow_keras/blob/main/rumors_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder 

In [2]:
tf.__version__

'2.9.2'

In [3]:
path="/content/truedata.csv"

In [4]:
df=pd.read_csv(path)


In [15]:
path2="/content/check.csv"

In [16]:
# train_df = pd.read_csv("train.csv")
# train_df.head()


In [17]:
# train_df = pd.read_csv(path2)



In [18]:
# df2=pd.read_csv(path2)
df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [19]:
# df1=pd.read_csv(path2)


In [20]:
train_df = pd.read_csv(path2)#here we have the dataset we extracted
train_df.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [21]:
Y = train_df['label']

In [22]:
group1 = pd.DataFrame(train_df.groupby(['author']).size()).reset_index()
group1.head()

,author,0
0,# 1 NWO Hatr,17
1,-NO AUTHOR-,54
2,10 Habits That Will Make Your Life Easier &amp...,1
3,10 More Beautiful Images That Remind You We St...,1
4,10 Movies That Could Change Your Understanding...,1


In [23]:
group2 = pd.DataFrame(train_df.groupby(['author'])['label'].sum()).reset_index()
group2.head()

,author,label
0,# 1 NWO Hatr,17
1,-NO AUTHOR-,54
2,10 Habits That Will Make Your Life Easier &amp...,1
3,10 More Beautiful Images That Remind You We St...,1
4,10 Movies That Could Change Your Understanding...,1


In [24]:
group1.columns = ['author','count']
group1.sort_values(by=['count'], ascending=False).head()


,author,count
2944,Pam Key,243
3929,admin,193
1762,Jerome Hudson,166
724,Charlie Spiering,141
1857,John Hayward,140


In [25]:
group1[group1['author']=='Starkman']

,author,count
3518,Starkman,84


In [26]:
group2.sort_values(by=['label'], ascending=False).head()


,author,label
3929,admin,193
2939,Pakalert,86
1111,Eddy Lavine,85
3518,Starkman,84
1376,Gillian,82


In [27]:
merge_groups = pd.merge(group1,group2, on='author')
merge_groups.head()

,author,count,label
0,# 1 NWO Hatr,17,17
1,-NO AUTHOR-,54,54
2,10 Habits That Will Make Your Life Easier &amp...,1,1
3,10 More Beautiful Images That Remind You We St...,1,1
4,10 Movies That Could Change Your Understanding...,1,1


In [28]:
merge_groups['prob_fake'] = merge_groups['label']/merge_groups['count']
merge_groups['prob_fake'].describe()

count    4201.000000
mean        0.470771
std         0.499016
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: prob_fake, dtype: float64

In [29]:
merge_groups[merge_groups['author']=='Pam Key']['prob_fake']

2944    0.004115
Name: prob_fake, dtype: float64

In [30]:
len(train_df['author'].unique())

4202

In [31]:
train_df['title_lower'] = train_df["title"].str.lower()
train_df['title_no_punctuation'] = train_df['title_lower'].str.replace('[^\w\s]','')
train_df['title_no_punctuation'] = train_df["title_no_punctuation"].fillna("fillna")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [32]:
train_df['text_lower'] = train_df["text"].str.lower()
train_df['text_no_punctuation'] = train_df['text_lower'].str.replace('[^\w\s]','')
train_df['text_no_punctuation'] = train_df["text_no_punctuation"].fillna("fillna")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [33]:
train_df['author_lower'] = train_df["author"].str.lower()
train_df['author_no_spaces'] = train_df['author_lower'].str.replace(' ','_')

In [34]:
train_df['author_no_spaces'].head()

0         darrell_lucus
1       daniel_j._flynn
2    consortiumnews.com
3       jessica_purkiss
4        howard_portnoy
Name: author_no_spaces, dtype: object

In [35]:
max_features=5000 #we set maximum number of words to 5000
maxlen=400

In [36]:
tok = tf.keras.preprocessing.text.Tokenizer(num_words=max_features) #again tokenizer step

In [37]:
tok.fit_on_texts(list(train_df['text_no_punctuation'])+list(train_df['title_no_punctuation'])+list(train_df['author_no_spaces'].astype(str))) #fit to cleaned text

In [38]:
print(len(tok.word_index))
vocab_size = len(tok.word_index) + 1 
#this represents the number of words that we tokenize different from max_features but necessary for
#the definition of the dimension of the embedding space

216068


In [39]:
text_df = tok.texts_to_sequences(list(train_df['text_no_punctuation'])) #this is how we create sequences
text_df = tf.keras.preprocessing.sequence.pad_sequences(text_df, maxlen=maxlen) #let's execute pad step

In [40]:
title_df = tok.texts_to_sequences(list(train_df['title_no_punctuation'])) #this is how we create sequences
title_df = tf.keras.preprocessing.sequence.pad_sequences(title_df, maxlen=maxlen)

In [41]:
author_df = tok.texts_to_sequences(list(train_df['author_no_spaces'].astype(str))) #this is how we create sequences
author_df = tf.keras.preprocessing.sequence.pad_sequences(author_df, maxlen=maxlen)

In [42]:
train_df = author_df 

In [43]:
from sklearn.model_selection import train_test_split #divide into train and test set

In [44]:
X_train, X_test, y_train, y_test = train_test_split(train_df, Y, test_size=0.1, random_state=42)

In [45]:
embedding_dim = 50 #this is the final dimension of the embedding space.

In [46]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(input_dim=vocab_size, #embedding input
                           output_dim=embedding_dim,#embedding output
                           input_length=maxlen), #maximum length of an input sequence
  tf.keras.layers.Flatten(), #flatten layer

  tf.keras.layers.Dense(1, activation=tf.nn.sigmoid) #no more softmax

])

In [47]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',#no more categorical_crossentropy
              metrics=['accuracy'])

In [48]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 50)           10803450  
                                                                 
 flatten (Flatten)           (None, 20000)             0         
                                                                 
 dense (Dense)               (None, 1)                 20001     
                                                                 
Total params: 10,823,451
Trainable params: 10,823,451
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.fit(np.array(X_train), np.array(y_train), epochs=1)

585/585 [==============================] - 66s 112ms/step - loss: 0.4631 - accuracy: 0.7709


In [50]:
model.evaluate(np.array(X_test), np.array(y_test)) 

65/65 [==============================] - 0s 3ms/step - loss: 0.3670 - accuracy: 0.8173


[0.36700886487960815, 0.817307710647583]